In [1]:
import torch
from torch_geometric import datasets
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader 

from models import VGAE

%load_ext autoreload


In [2]:
%autoreload 2

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = datasets.Planetoid(root='/__data/Cora', name='Cora', split='public')

In [6]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [4]:
transform = RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=False, add_negative_train_samples=True)

In [5]:
train_data, val_data, test_data = transform(dataset[0])

In [6]:
print('train data', train_data)
print('val data', val_data)
print('test data', test_data)

dataset[0].edge_index.shape[1] == train_data.edge_label_index.size(1) + val_data.edge_label_index.size(1)  + test_data.edge_label_index.size(1) 

train data Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[8976], edge_label_index=[2, 8976])
val data Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[526], edge_label_index=[2, 526])
test data Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054])


True

In [7]:
feature_dim = dataset[0].x.shape[1]
print('input dim', feature_dim)

input dim 1433


In [14]:
# check forward pass
n_vgae = VGAE(input_dim=feature_dim, latent_dim=16, dropout=0.0, latent_distr='normal').to(device)

out_n_vgae, mus_n_vgae, logsigmas2s_n_vgae  = n_vgae(val_data.x.to(device), val_data.edge_label_index.to(device))
print('out', out_n_vgae.shape)
print('mus', mus_n_vgae.shape)
print('logsigmas2s', logsigmas2s_n_vgae.shape)

out torch.Size([2708, 2708])
mus torch.Size([2708, 16])
logsigmas2s torch.Size([2708, 16])


In [13]:
s_vgae = VGAE(input_dim=feature_dim, latent_dim=16, dropout=0.0, latent_distr='vMF').to(device)

out_s_vgae, mus_s_vgae, logsigmas2s_s_vgae  = s_vgae(val_data.x.to(device), val_data.edge_label_index.to(device))
print('out', out_s_vgae.shape)
print('mus', mus_s_vgae.shape)
print('logsigmas2s', logsigmas2s_s_vgae.shape)

HKHKKH
Z torch.Size([2708, 16])
tensor([[-0.0847, -0.1055, -0.2225,  ...,  0.2481,  0.1600, -0.2919],
        [ 0.2687, -0.3662,  0.0672,  ..., -0.4139, -0.2646,  0.0187],
        [-0.1024,  0.1328,  0.3487,  ..., -0.0356,  0.0399, -0.2639],
        ...,
        [-0.3298, -0.1644, -0.0432,  ..., -0.0693, -0.7771, -0.0525],
        [ 0.1257, -0.0194, -0.4285,  ...,  0.3678, -0.2338,  0.0298],
        [ 0.1344, -0.1018,  0.2154,  ..., -0.2153,  0.5185, -0.0746]],
       dtype=torch.float64, grad_fn=<StackBackward0>)
out torch.Size([2708, 2708])
mus torch.Size([2708, 16])
logsigmas2s torch.Size([2708, 1])


In [11]:
t = torch.ones(1, 3)
torch.squeeze(t).shape

torch.Size([3])

In [29]:
from scipy.stats import beta
b = torch.tensor([1., 6., 8.], requires_grad=True)
eps = beta.rvs((16 -1)/2, (16 -1)/2, size=1)
print(eps)
(1 - (1+b)*eps[0]) / (1 - (1-b)*eps[0])


[0.50104177]


tensor([-0.0021, -0.7153, -0.7786], grad_fn=<DivBackward0>)